---
title: Generic Programming
abstract: |
    Generic programming in C++ is a paradigm that enables writing flexible, reusable code by abstracting types and operations through templates. A *template class* defines a blueprint for classes parameterized by types, while a *template function* allows functions to operate on different types without rewriting code. Templates can have *type parameters* or *non-type parameters*, which represent compile-time constants. Templates can also be variadic using *parameter packs*, which allow an arbitrary number of parameters/arguments of different types to be specified in different contexts. Parameter pack can be expanded according to different patterns, for instance, using the *folding expressions*.

---

In [ ]:
from __init__ import *
!mkdir -p private

In [ ]:
%%cpp
#include "utility.hpp"

In [ ]:
if not input('Load JupyterAI? [Y/n]').lower()=='n':
    %reload_ext jupyter_ai

## Motivation

Consider implementing the Python function `print` in C++:

In [ ]:
print?

1. `*` is an unpacking operator in Python. In `*args`, `args` is a parameter pack that gets unpacked to a variable number of parameters, which takes the values of the positional arguments:

In [ ]:
print(1, 2.0, 'c', "def") 

2. `sep=' '` and `end='\n'` are optional key-only parameters which are specified as follows:

In [ ]:
print(1, 2 ,3, sep=", ", end=";\n")

3. `file` allows one to print to a file-like object:

In [ ]:
with open('private/python_output.txt', 'w') as f:
    print("Hello,", "World!", file=f)

We will learn how to implement `print` in C++.

## Variadic Template

### Parameter Pack

**How to define a function that takes a variable number of arguments in C++?**

Similar to `*` in Python, we can use `...` to define a [parameter pack](https://en.cppreference.com/w/cpp/language/parameter_pack.html):

In [ ]:
%%writefile private/pack.cpp
#include <iostream>

using std::cout;

template <auto... pack>
auto foo() {
    return sizeof...(pack);
}

int main() {
    cout << foo<1, 2.0, 'c'>() << '\n';                  
    return 0;
}

In [ ]:
!cd private && g++ -std=c++20 pack.cpp -o pack && ./pack

In the above definition
- `auto... pack` is a named [constant template parameter](http://en.cppreference.com/w/cpp/language/template_parameters.html#Constant_template_parameter) 
  ```cpp
  type ... name (optional)
  ```
  which can pack a variable number of non-type constants of structural type.
- [`sizeof...`](https://en.cppreference.com/w/cpp/language/sizeof....html) is an operator that gives the number of arguments of the named pack `args`.

However, many types are not structural types, and we often want a function to take mutable arguments. For instance:

1. ```cpp
   cout << foo<"def">() << '\n';
   ```
   fails because `"def"` is not a valid template argument for type `const char*`. String literals cannot be used in this context.

2. ```cpp
   const int a=rand();
   cout << foo<a>() << '\n';  
   ```
   fails because `a` is not a constant expression.

In [ ]:
%%ai
Explain in a paragraph or two why the type for constant template parameter 
must be a structural type?

In [ ]:
%%writefile private/pack.cpp
#include <iostream>

using std::cout;

template <class... Ts>
auto foo(Ts... args) {
    return sizeof...(args);
}

int main() {
    cout << foo(1, 2.0, 'c', "def") << '\n';
    return 0;
}

In [ ]:
!cd private && g++ -std=c++20 pack.cpp -o pack && ./pack

In the above definition, `class... Ts` is a named [type template parameter](http://en.cppreference.com/w/cpp/language/template_parameters.html#Type_template_parameter) 
```cpp
type-parameter-key ... name (optional)
```
where type-parameter-key is either `class`, or equivalently, `typename`. It packs a variable number of types, which is used in `Ts... args` to specify a named [function parameter pack](https://en.cppreference.com/w/cpp/language/parameter_pack.html#Function_parameter_list)
```cpp
pack-name ... pack-param-name (optional)
```
which allows the functions to take a variable number of arguments of possibly different types.

### Pack Expansion

**How to implement the `print` function?**

Unfortunately, we cannot index a pack prior to the standard [`C++26`](https://en.cppreference.com/w/cpp/language/pack_indexing.html).

In [ ]:
%%ai
Explain in a paragraph or two why elements of a parameter pack cannot be 
extracted using subscription operation in C++.

Nevertheless, we can access individual argument of a pack by pack expansion and recursion:

In [ ]:
%%writefile private/print.cpp
#include <iostream>

using std::cout;

void print() {
    cout << '\n';
}

template <class T, class... Ts>
void print(const T &first, const Ts &... rest) {
    cout << first;
    if (sizeof...(rest)) cout << ' ';
    print(rest...);
}

int main() {
    print(1, 2.0, 'c', "def");
    return 0;
}

In [ ]:
!cd private && g++ -std=c++20 print.cpp -o print && ./print

- The base case with no argument is handled by
  ```cpp
  void print() {
      cout << '\n';
  }
  ```

- The recursive call `print(rest...)` uses a [function argument list](https://en.cppreference.com/w/cpp/language/parameter_pack.html#Function_argument_lists), which is a simple [pack expansion](https://en.cppreference.com/w/cpp/language/parameter_pack.html#Pack_expansion)
  ```cpp
  pattern ...
  ```
  that expands the pattern (containing at least one pack) into zero or more instantiations of the pattern, where the name of each pack is replaced by each of the elements from the pack, in order.  
  E.g., for the call `print(1, 2.0, 'c', "def");`, the recursive call is expanded to
  ```cpp
  print(
      2.0, 'c', "def"  // rest...
  );
  ```

- In the function parameter pack `const Ts&... rest`, pack expansion is also applied to `const Ts &...` to expand the template parameter pack `class... Ts`.  
  E.g., for the call `print(1, 2.0, 'c', "def")`, the compiler expands the function parameter pack as follows:
  ```cpp
  void print(
      const int &,                                    // const T &first,
      const double &, const char &, const char (&)[4] // const Ts&... rest
  );
  ```

Pack expansion can be used in [various contexts](https://en.cppreference.com/w/cpp/language/parameter_pack.html#Expansion_loci). E.g., a simpler implementation of `print` is as follows:

In [ ]:
%%cpp
void print(const auto &... args) {
    ((cout << args << ' '), ...);
    cout << "\b\n";
};

print(1, 2.0, 'c', "def");

`void print(const auto &... args)` is an [abbreviated function template](https://en.cppreference.com/w/cpp/language/function_template.html#Abbreviated_function_template) equivalent to 
```cpp
template <class Ts>
void print(const Ts &... args)
```

`((cout << args << ' '), ...)` is  a [unary right fold expression](https://en.cppreference.com/w/cpp/language/fold.html)
```cpp
(E op ...)
```
which expands to
```cpp
(E1 op (... op (EN-1 op EN)))
```
where `E1, E2, ..., EN` are the instantiations of the pattern `E`.

::::{exercise}
:label: ex:urf

Give the pack expansion of `((cout << args << ' '), ...)` for the function call `print(1, 2.0, 'c', "def")`.

::::

YOUR ANSWER HERE

An alternative implementation of `print` in one line is:

In [ ]:
%%cpp
void print(const auto &... args) {
    ((cout << args << ' '), ..., (cout << "\b\n"));
};

print(1, 2.0, 'c', "def");

`((cout << args << ' '), ..., (cout << "\b\n"));` is  a binary right fold expression

```cpp
(E op ... op I)
```
which expands to
```cpp
(E1 op (... op (EN op I)))
```
where `E1, E2, ..., EN` are the instantiations of the pattern `E`.

::::{exercise}
:label: ex:sum

Complete the following function using a binary left fold expression to add a variable number of arguments together. `sum()` should return `0`.

::::

In [ ]:
%%cpp
auto sum(const auto &... args) {
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
}

cout << sum(1, 2, 3, 4) << '\n'; // 10 

In C, there is an alternative way of defining a function with a variable number of arguments using the so-called [variadic arguments](https://en.cppreference.com/w/cpp/language/variadic_arguments.html):

In [ ]:
%%cpp
int add_nums(int count...) // int add_nums(int count, ...)
{
    int result = 0;
    std::va_list args;
    va_start(args, count);
    for (int i = 0; i < count; ++i)
        result += va_arg(args, int);
    va_end(args);
    return result;
}
 
std::cout << add_nums(4, 25, 25, 50, 50) << '\n';

[The above code](https://en.cppreference.com/w/cpp/utility/variadic/va_start.html#Example) is not a variadic template because the ellipsis in `int add_nums(int count, ...)` is a separate parameter rather than a part of a parameter declarator as in `auto sum(const auto &... args)`.

::::{caution} Variadic templates vs arguments

Variadic templates in C++ is an improvement on variadic arguments in C, allowing developers to write functions, classes, and data structures that accept an arbitrary number of arguments while retaining full type safety and compile‑time checks. In comparison, the variadic arguments uses the [C `stdarg` mechanism](https://en.cppreference.com/w/cpp/header/cstdarg.html), which relies on preprocessor macros and runtime reinterpretation of arguments with no type information, variadic templates preserve the exact types of each argument.

::::

In [ ]:
%%ai
Explain in a paragraph or two how variadic template is different from variadic
arguments in C++?

## Template Class

Recall that the python `print` function has the optional parameters `sep` and `end` for the string inserted between values and that appended to the last value respectively:

In [ ]:
print(1, 2, 3, sep=", ", end=";\n")

**How to take `sep` and `end` as the optional parameters?**

::::{caution} The challenges...

- There is no such thing as keyword argument in C++, i.e., we cannot have the function call
  ```cpp
  print(1, 2, 3, sep=", ", end=";\n");
  ```
- A parameter pack cannot come before a parameter, i.e., we cannot declare
  ```cpp
  void print(const auto &... args, const string &sep, const string &end);
  ```
- An optional parameter cannot come before a required parameter, i.e., we cannot declare
  ```cpp
  void print(const string &sep=", ", const string &end";\n", const auto &... args);
  ```
::::

While C++ doesn’t support keyword arguments natively, you can achieve similar flexibility using a [fluent interface pattern](https://en.wikipedia.org/wiki/Fluent_interface):

In [ ]:
%%cpp
#include "utility.hpp"

auto &print=printer<>::get_print();

print(1, 2, 3);
print.sep(", ")(4, 5, 6);
print.end(";\n")(7, 8, 9);
print.sep(", ").end(";\n")(1, 2, 3);

`printer<>::get_print()` is defined in [](./utility.hpp) to create a `printer` object that uses method chaining to set `sep` and `end`. `print` is a `functor` that has a variadic template method to print a variable number of arguments.

To illustrate the idea, consider the following simplified implementation:

In [ ]:
%%writefile private/Printer.cpp
#include <iostream>

using std::cout;

class Printer final {
    const char *sep_, *end_;

    Printer(const char *sep=" ", const char *end="\n")
    : sep_(sep), end_(end) {}

    Printer(const Printer &) = delete;
    Printer(Printer &&) = delete;
    Printer& operator=(const Printer&) = delete;
    Printer& operator=(Printer&&) = delete;

public:
    const static Printer &get_print() {
        const static Printer print;
        return print;
    }

    Printer sep(const char* s) const & { return Printer(s, end_); }
    Printer end(const char* s) const & { return Printer(sep_, s); }

    Printer &sep(const char* s) && { return sep_=s, *this; }
    Printer &end(const char* s) && { return end_=s, *this; }

    void operator()() const { cout << end_; }
    void operator()(const auto &first, const auto &... rest) const {
        cout << first;
        ((cout << sep_ << rest), ..., (cout << end_));
    }
};

// test
#include <iostream>

int main() {
    auto &print=Printer::get_print();
    print.sep(", ").end(";\n")(1, 2, 3);
    print(1, 2, 3);
    return 0;
}

In [ ]:
!cd private && g++ -std=c++20 Printer.cpp -o Printer && ./Printer

The code defines a functor class with variadic template to print a variable number of arguments:

```cpp
class Printer final {
...
public:
    ...
    void operator()() const { cout << end_; }
    void operator()(const auto &first, const auto &... rest) const {
        cout << first;
        ((cout << sep_ << rest), ..., (cout << end_));
    }
};
```

::::{caution}

Due to an issue with the `ROOT` kernel, a class that contains a template method needs to be declared as a template class such as

```cpp
template <auto...>
class Printer final {
...
};
```

in which case `Printer::get_print()` should be called as `Printer<>::get_print()` instead.

::::

::::{exercise}
:label: ex:sep

Why not combine the two overloads into one as follows?

```cpp
void operator()(const auto &... args) {
    ((cout << sep_ << args), ..., (cout << `\b` << end_));
};
```

::::

YOUR ANSWER HERE

`Printer` uses the [singleton design pattern](https://en.wikipedia.org/wiki/Singleton_pattern) to define a unique static functor `print`:

- `get_print()` uses a local static, which is efficient and thread-safe for initialization.
  ```cpp
  const static Printer &get_print() {
      const static Printer print;
      return print;
  }
  ```

- Hidden/deleted constructors/assignments prevents unnecessary copies/instantiation, which is good for performance and correctness.
  ```cpp
  ...
  class Printer final {
      const char *sep_, *end_;
      
      Printer(const char *sep=" ", const char *end="\n")
      : sep_(sep), end_(end) {}
  
      Printer(const Printer &) = delete;
      Printer(Printer &&) = delete;
      Printer& operator=(const Printer&) = delete;
      Printer& operator=(Printer&&) = delete;
  ...
  ```
  The `class` is also specified to be [`final`](https://en.cppreference.com/w/cpp/language/final.html), which prevents inheritance, ensuring that there is no derived class that may mutate the private members.

::::{exercise}
:label: ex:ref-qualifier

Explain why `sep()` and `end()` are overloaded with [`&` and `&&` ref-qualifiers](https://en.cppreference.com/w/cpp/language/member_functions.html#Member_functions_with_ref-qualifier):

```cpp
...
    Printer sep(const char* s) const & { return Printer(s, end_); }
    Printer end(const char* s) const & { return Printer(sep_, s); }

    Printer &sep(const char* s) && { return sep_=s, *this; }
    Printer &end(const char* s) && { return end_=s, *this; }
...
```

::::

YOUR ANSWER HERE

::::{exercise}
:label: ex:Printer1

In `print.sep(", ").end(";\n")(1, 2, 3);`, which overloads does `.sep(", ")` and `.end(";\n")` respectively resolve to?

::::

YOUR ANSWER HERE

::::{exercise}
:label: ex:string

Explain why the following code fail?

```cpp
print.sep(std::string(4, '-'))(1, 2, 3);
```

::::

::::{solution} ex:string
:class: dropdown

It fails because there is no matching function for call to `Printer::sep(std::string) const`. The argument of `sep` is `const char *`.

::::

**How to extend `Printer` to support string containers for `sep` and `end`?**

One way is to create a template class:

```cpp
template <class StringT = std::string> // default template argument
class Printer final {
    /* original
    const char *sep_, *end_;
    */
    StringT sep_, end_;

    /* original
    Printer(const char *sep=" ", const char *end="\n")
    : sep_(sep), end_(end) {}
    */
    Printer(StringT sep = " ", StringT end = "\n")
    : sep_(std::move(sep)), end_(std::move(end)) {}

    Printer(const Printer&) = delete;
    Printer(Printer&&) = delete;
    Printer& operator=(const Printer&) = delete;
    Printer& operator=(Printer&&) = delete;

public:
    const static Printer& get_print() {
        const static Printer print;
        return print;
    }

    Printer sep(const StringT& s) const& { return Printer(s, end_); }
    Printer end(const StringT& s) const& { return Printer(sep_, s); }

    /* original
    Printer &sep(const char* s) && { return sep_=s, *this; }
    */
    template <class T>
    Printer& sep(T&& s) && {
        sep_ = StringT(std::forward<T>(s));
        return *this;
    }
    /* original
    Printer &end(const char* s) && { return end_=s, *this; }
    */
    template <class T>
    Printer& end(T&& s) && {
        end_ = StringT(std::forward<T>(s));
        return *this;
    }

    void operator()() const { cout << end_; }
    void operator()(const auto &first, const auto &... rest) const {
        cout << first;
        ((cout << sep_ << rest), ..., (cout << end_));
    }
};
```

In [ ]:
%%cpp
template <class StringT = std::string>
class Printer final {
    StringT sep_, end_;

    Printer(StringT sep = " ", StringT end = "\n")
    : sep_(std::move(sep)), end_(std::move(end)) {}

    Printer(const Printer&) = delete;
    Printer(Printer&&) = delete;
    Printer& operator=(const Printer&) = delete;
    Printer& operator=(Printer&&) = delete;

public:
    const static Printer& get_print() {
        const static Printer print;
        return print;
    }

    Printer sep(const StringT& s) const& { return Printer(s, end_); }
    Printer end(const StringT& s) const& { return Printer(sep_, s); }

    template <class T>
    Printer& sep(T&& s) && {
        sep_ = StringT(std::forward<T>(s));
        return *this;
    }
    template <class T>
    Printer& end(T&& s) && {
        end_ = StringT(std::forward<T>(s));
        return *this;
    }

    void operator()() const { cout << end_; }
    void operator()(const auto &first, const auto &... rest) const {
        cout << first;
        ((cout << sep_ << rest), ..., (cout << end_));
    }
};

// std::string
auto &print=Printer<>::get_print();
print.sep(string(4, '-')).end(";\n")(1, 2, 3);

The initialization `auto &print=Printer<>::get_print();` instantiates the template class 

```cpp
template <class StringT = std::string> // default template argument
```

with the template parameter `StringT` equal to the [default template argument](https://en.cppreference.com/w/cpp/language/template_parameters.html#Default_template_arguments) `std::string`.

`get_print()` then creates the static functor `print` with
```cpp
...
    /* original
    const char *sep_, *end_;
    */
    StringT sep_, end_;

    /* original
    Printer(const char *sep=" ", const char *end="\n")
    : sep_(sep), end_(end) {}
    */
    Printer(StringT sep = " ", StringT end = "\n")
    : sep_(std::move(sep)), end_(std::move(end)) {}
...
```
where `sep_` and `end_` have a generic type `StringT`, which works for string containers, instead of `const char *`, which only works for C-string.

::::{exercise}
:label: ex:move

Explain why `std::move` is added to the member initializer list.

::::

::::{solution} ex:move
:class: dropdown

`std::move` is added in the member initializer list to enable move semantics, which avoids unnecessary deep copies when initializing members from constructor parameters. In the original version, `sep_(sep)` and `end_(end)` is efficient for C-strings because only the memory addresses, not the arrays of characters, are copied.

::::

::::{exercise}
:label: ex:forward

Explain why `std::forward<T>` is used in the following code:

```cpp
...
    /* original
    Printer &sep(const char* s) && { return sep_=s, *this; }
    */
    template <class T>
    Printer& sep(T&& s) && {
        sep_ = StringT(std::forward<T>(s));
        return *this;
    }
    /* original
    Printer &end(const char* s) && { return end_=s, *this; }
    */
    template <typename T>
    Printer& end(T&& s) && {
        end_ = StringT(std::forward<T>(s));
        return *this;
    }
...
```

YOUR ANSWER HERE

**Can we also use other output streams such as `std::ofstream` from `<fstream>`?**

We can add another template parameter for the output stream:

::::{exercise}
:label: ex:ostream1

Complete the following code by adding a new template parameter `StreamT` with a default type of `std::ostream`:
- `get_print` should return a static functor named `print`, which writes to `std::cout` by default.
- Add different overloads for the method `stream` to configure the output stream, similar to how `sep` and `end` configures their respective values.

::::

In [ ]:
%%cpp
/* original
template <class StringT = std::string>
*/
template <class StringT = std::string, class StreamT = std::ostream>
class Printer final {
    StringT sep_, end_;
    StreamT* out_; // +

    /* original
    Printer(StringT sep = " ", StringT end = "\n")
    : sep_(std::move(sep)), end_(std::move(end)) {}
    */
    Printer(StringT sep = " ", StringT end = "\n", StreamT* out = &std::cout)
    : sep_(std::move(sep)), end_(std::move(end)), out_(out) {}    

    Printer(const Printer&) = delete;
    Printer(Printer&&) = delete;
    Printer& operator=(const Printer&) = delete;
    Printer& operator=(Printer&&) = delete;

public:
    const static Printer& get_print() {
        const static Printer print;
        return print;
    }

    template <class T>
    Printer& sep(T&& s) && {
        sep_ = StringT(std::forward<T>(s));
        return *this;
    }
    template <class T>
    Printer& end(T&& s) && {
        end_ = StringT(std::forward<T>(s));
        return *this;
    }
    /*
    # REPLACE THE ENTIRE COMMENT WITH YOUR CODE #
    */
};

// std::string
auto &print=Printer<>::get_print();
print.sep(string(4, '-')).end(";\n")(1, 2, 3);

// Use a file stream
std::ofstream file("private/output.txt");
print.stream(file).sep(" -> ").end("\n")("File", "Output");
file.close();

In [ ]:
!cat private/output.txt
# File -> Output

::::{exercise}
:label: ex:ostream2

Why is it a good idea to 
1. declare `StringT sep_, end_;` instead of `StringT *sep_, *end_;`, but
2. `StreamT* out_;` instead of `StreamT out_;`?

::::

YOUR ANSWER HERE